In [1]:
import scrapy
import requests

from scraper.spiders.stalker import get_urls

In [2]:
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
BASE_URL = 'http://apps.carleton.edu'
DIR_BASE_URL = BASE_URL + '/campus/directory'

In [3]:
urls = get_urls()
url = urls[0]
url

'http://apps.carleton.edu/campus/directory/?last_name=A&year=2018'

In [33]:
# r = requests.get(url)
# response = scrapy.http.TextResponse(r.url, body=r.text, encoding='utf-8')

# with open ('campus.html', 'wb') as f:
#     f.write(response.body)

In [4]:
with open('campus.html', 'r') as f:
    body = f.read()
    
response = scrapy.http.TextResponse(url, body=body, encoding='utf-8')

In [5]:
people = response.css('li.person')
person = people[1]

In [6]:
person

<Selector xpath="descendant-or-self::li[@class and contains(concat(' ', normalize-space(@class), ' '), ' person ')]" data='<li class="person">\n<div class="heading '>

In [7]:
name = (person.css('h2::text').extract_first() or
        person.css('h2 a::text').extract_first())

In [42]:
name.split(' ', 1)

['Alex', 'Aeppli']

In [8]:
def scrape_person(person):
    """Returns dict of scraped data for given person"""

    name = (person.css('h2::text').extract_first() or
            person.css('h2 a::text').extract_first())
    
    first_name, last_name = name.split(' ', 1)

    majors = person.css('span.majors span.major::text').extract()
    majors = [m.replace('\u200b', '') for m in majors]  # 0-width space

    minors = person.css('span.majors span.minor::text').extract()
    minors = [m.replace('\u200b', '') for m in minors]  # 0-width space

    year = person.css('span.affiliation::text').extract_first()
    img = BASE_URL + person.css('div.image img::attr(src)').extract_first()
    house = person.css('p.location a::text').extract_first()
    tags = person.css('div.tags a::text').extract()
    email = person.css('div.email a::text').extract_first()

    return dict(
        first_name=first_name, last_name=last_name, year=year,
        img=img, majors=majors, minors=minors,
        house=house, tags=tags, email=email)

In [9]:
data = []

for p in people:
    data.append(scrape_person(p))

In [10]:
data

[{'first_name': 'Alex',
  'last_name': 'Adamczyk',
  'year': '2018',
  'img': 'http://apps.carleton.edu/stock/ldapimage.php?id=adamczyka&source=campus_directory',
  'majors': ['Political Science/International Relations'],
  'minors': [],
  'house': 'Brooks House 200B',
  'tags': ['English',
   'Live Music',
   'Political Science',
   'Law School',
   'Environmental Law',
   'Music Promotion',
   'Museums',
   'Consulting/Finance',
   'Business Consulting'],
  'email': 'adamczyka@carleton.edu'},
 {'first_name': 'Alex',
  'last_name': 'Aeppli',
  'year': '2018',
  'img': 'http://apps.carleton.edu/stock/ldapimage.php?id=aepplia&source=campus_directory',
  'majors': ['Physics'],
  'minors': [],
  'house': 'Eugster House 100C',
  'tags': [],
  'email': 'aepplia@carleton.edu'},
 {'first_name': 'Wisdom',
  'last_name': 'Akpan',
  'year': '2018',
  'img': 'http://apps.carleton.edu/stock/ldapimage.php?id=akpanw&source=campus_directory',
  'majors': ['Chemistry'],
  'minors': [],
  'house': 'Owe

In [41]:
import json

In [15]:
with open('data.json', 'w') as f:
    json.dump(data, f)